In [5]:
import numpy as np 
import pandas as pd 
import os
import torch
import librosa
from torch.utils.data import Dataset, DataLoader

# memory
from my_memory_profiler import profile

In [6]:
@profile
def label_generator(DATA_PATH_NeuroV):
    data_list = []
    
    for dirname, _, filenames in os.walk(DATA_PATH_NeuroV):
        for filename in filenames:
            if filename.endswith('.wav'):  # Check if the file is a WAV file
                identifiers = filename.split('.')[0].split('_')
                Speaker_ID = int(identifiers[-1])
                Label = 1 if identifiers[0] == 'PD' else 0
                data_list.append({
                    "Speaker_ID": Speaker_ID,
                    "Label": Label,
                    "FileName": filename
                })
    
    data_NeuroV = pd.DataFrame(data_list)
        
    return data_NeuroV

In [7]:
@profile
def num_seg(data_dir, audio_labels, SAMPLE_RATE, output_csv=None, filenames=False):
    time_leng=0.4
    sample_leng=int(time_leng*SAMPLE_RATE)
    overloap=2
    segments_info=[]
    # audio_labels=pd.read_csv('labels.csv')


    #Processs data to train
    for data_ind, audio_label in enumerate(audio_labels.values):
        file_name=audio_label[-1]
        data_path=os.path.join(data_dir, file_name)
        audio, sample_rate = librosa.load(data_path, sr=SAMPLE_RATE)
        audio_len=len(audio)
        #print(audio_len)
        #plt.plot(audio)
        audio=audio/np.max(abs(audio))
        indx=[i for i,x in enumerate(np.sqrt(abs(audio))) if x>.30]
        segments=0
        if (indx[0]+sample_leng)<audio_len:
            for i in range(int((-indx[0]+indx[len(indx)-1])/(sample_leng/overloap))):
                ind_start = i * int(sample_leng/overloap)+indx[0]
                ind_end = ind_start + sample_leng
                if ind_end <= indx[-1]:
                    segments+=1
            if filenames:
                segments_info.append({
                    'Filename': file_name,
                    'Segments': segments})
            else: 
                segments_info.append({'Segments':segments})
            
            print(" Processed {}/{} files".format(data_ind,2902),end='\n')
        else:
            print(" Processed {}/{} files".format(data_ind,2902),end='\n')
            if filenames:
                segments_info.append({
                    'Filename': file_name,
                    'Segments': segments})
            else: 
                segments_info.append({'Segments':segments})
    

    df_segments = pd.DataFrame(segments_info)
    if(output_csv is None):
        return df_segments
    else:
        df_segments.to_csv(output_csv, index=False)
        
    

In [8]:
SAMPLE_RATE_=44100
DATA_PATH_NeuroV = 'neurovoz/zenodo_upload/audios/'

In [11]:
audio_labels=label_generator(DATA_PATH_NeuroV)
audio_labels.head()

label_generator: consumed memory: 0.000000 GB


,Speaker_ID,Label,FileName
0,16,1,PD_PERRO_0016.wav
1,143,0,HC_O2_0143.wav
2,78,1,PD_CARMEN_0078.wav
3,81,0,HC_PERRO_0081.wav
4,52,0,HC_PATATA_BLANDA_0052.wav


In [ ]:
numero_segmentos=num_seg(DATA_PATH_NeuroV, audio_labels, SAMPLE_RATE_)
numero_segmentos.head()

In [ ]:
numero_de_segmentos=pd.read_csv('prueba.csv')
numero_de_segmentos.head()

,Filename,Segments
0,PD_PERRO_0016.wav,8
1,HC_O2_0143.wav,20
2,PD_CARMEN_0078.wav,7
3,HC_PERRO_0081.wav,7
4,HC_PATATA_BLANDA_0052.wav,9


In [ ]:
number=numero_de_segmentos['Segments'].sum()
print("There are {} segments".format(number))

There are 59379 segments
